Testing the voting scheme on a -ve mode file

In [36]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
from scipy import stats

base_dir = '/Users/simon/git/ms1fun/'
sys.path.append(base_dir + 'code')
sys.path.append(base_dir + 'dbs')

from corr_cluster import Peak,BetaLike,CorrCluster
from formula import Formula
import pylab as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
files = []

prefix = '/Users/simon/Downloads/Beer3_mzXML_mzML_files/PositiveMode/Beer3Full/csv/'
filename = 'Beer_3_Full1'
files.append((prefix,filename,False))

In [38]:
import transformation
transformations = transformation.load_from_file(base_dir + 'dbs/neg_transformations.yml')
print "Loaded " + str(len(transformations)) + " transformations"

Loaded 72 transformations


In [39]:
bl = BetaLike()
clusterings = {}
for f in files:

    filename = f[1]
    prefix = f[0]
    print "Running file " + filename
    csvfile = prefix + filename + '.csv'
    matfile = prefix + filename + '.corr.mat'
    signalfile = prefix + filename + '.peakml.signal'
    # Simon is an idiot...remember to change 'correct = True' or 'correct = False' 
    # coc = CorrCluster(bl,csvfile,matfile,signal_file=signalfile,greedy_thresh=0.7,correct=True)
    coc = CorrCluster(bl,csvfile,matfile,signal_file=signalfile,greedy_thresh=0.7,correct=f[2])
    clusterings[filename] = coc

Running file Beer_3_Full1
1421.38000488
Loaded 7863 peaks
Reading shape correlations from /Users/simon/Downloads/Beer3_mzXML_mzML_files/PositiveMode/Beer3Full/csv/Beer_3_Full1.corr.mat
Greedy clustering done, resulting in 1116 clusters


In [40]:
from voter import Voter,PeakGroup
groups = {}
for f in files:
    v = Voter(transformations)
    filename = f[1]
    print "Performing voting on file {}".format(filename)
    file_groups = []
    for cluster in clusterings[filename].clusters:
        file_groups += v.make_groups(cluster.members)
    groups[filename] = file_groups

Performing voting on file Beer_3_Full1


In [41]:
PROTON = 1.00727645199076


class Mol(object):
    def __init__(self,name,formula,mass,rt):
        self.name = name.strip()
        self.formula = formula.strip()
        self.mass = mass
        self.rt = rt
    def __str__(self):
        return "{} ({},{})".format(self.name,self.mass,self.rt)
    def __repr__(self):
        return "{} ({},{})".format(self.name,self.mass,self.rt)

In [42]:
std_file = base_dir + 'dbs/std1_20130822_130526.csv'
std_file = '/Users/simon/Downloads/Beer3_mzXML_mzML_files/Std1_1_20150422_150810_combined.csv'
mols = []
with open(std_file,'rU') as f:
    for i in range(9):
        f.readline() # remove heads
    for line in f:
        split_line = line.split(',')
        polarity = split_line[4]
        rts = split_line[9] # observed, not predicted
        if rts == '-':
            rt = 0.0
        else:
            rt = float(rts)
        if polarity == '+' and rt > 0.0:
            name = split_line[2]
            formula = split_line[3]
            f = Formula(formula)
            new_mol = Mol(name,formula,f.compute_exact_mass(),rt*60.0)
            mols.append(new_mol)
mols = sorted(mols,key = lambda x: x.mass)
for mol in mols:
    print mol

1_3-Diaminopropane (74.0843983319,738.6)
Glycine (75.0320284101,658.8)
1-Aminopropan-2-ol (75.0684139166,938.4)
Mercaptoethanol (78.0139355049,1307.4)
beta-Alanine (89.0476784744,646.2)
L-Alanine (89.0476784744,633.6)
Glycerol (92.0473441234,480.0)
Phenol (94.0418648149,250.8)
1-Aminocyclopropane-1-carboxylate (101.047678474,569.4)
2-Aminobutan-4-olide (101.047678474,379.8)
4-Aminobutanoate (103.063328539,649.2)
Choline (103.099714045,915.0)
L-2_3-Diaminopropanoate (104.058577512,654.6)
L-Serine (105.042593097,657.6)
Phenylhydrazine (108.068748268,301.8)
cytosine (111.043261799,552.6)
Uracil (112.027277383,463.2)
Creatinine (113.058911863,496.8)
L-Proline (115.063328539,580.8)
Selenomethionine (117.078978603,538.8)
L-Valine (117.078978603,570.6)
Betaine (117.078978603,534.6)
L-2_4-Diaminobutanoate (118.074227576,743.4)
L-Threonine (119.058243161,619.2)
L-homoserine (119.058243161,637.8)
L-cysteine (121.019749164,612.6)
Nicotinamide (122.048012825,400.8)
Nicotinate (123.03202841,412.2)


In [43]:
def hit(m1,m2,rt1,rt2,mtol=20,rttol=1):
    if 1e6*np.abs(m1-m2)/m2 < mtol and np.abs(rt1-rt2)<rttol:
        return True
    else:
        return False
    
def get_hits(peaks,mols,transformations,mtol=20,rttol = 120.0):
    hits = {}
    for mol in mols:
        for peak in peaks:
            for t in transformations:
                if hit(t.transform(peak),mol.mass,peak.rt,mol.rt,mtol = mtol,rttol = rttol):
                    if mol in hits:
                        if np.abs(peak.rt - mol.rt) < np.abs(hits[mol].rt - mol.rt):
                            hits[mol] = peak
                    else:
                        hits[mol] = peak
    return hits

In [44]:
def get_M_hits(groups,mols,use_max_vote = True):
    hits = {}
    for g in groups:
        for mol in mols:
            if hit(mol.mass,g.M,mol.rt,g.rt,mtol=10,rttol=10):
                if not mol in hits:
                    hits[mol] = g
                else:
                    if use_max_vote:
                        if g.vote > hits[mol]:
                            hits[mol] = g
                    else:
                        current = hits[mol]
                        if np.abs(g.rt - mol.rt) < np.abs(current.rt - mol.rt):
                            hits[mol] = g
                
    return hits

In [45]:
all_hits = {}
for f in files:
    filename = f[1]
    all_hits[filename] = get_M_hits(groups[filename],mols,use_max_vote = True)

    

In [60]:
# Sort the groups according to votes (top ones first)
for f in files:
    filename = f[1]
    outpre = filename + '_NEG'

    temp_groups = sorted(groups[filename],key = lambda x:x.vote,reverse=True)
    outfile = outpre + '_by_vote.txt'
    

    with open(outfile,'w') as f:
        for i,group in enumerate(temp_groups):
            line = "vote: {}, M: {}\n".format(group.vote,group.M)
            f.write(line)
            head_line = '\tPeak m/z,Peak rt,Peak intensity,transformation (transformed mass,vote)\n'
            f.write(head_line)
            for (peak,transformation,transmass) in sorted(group.members,key = lambda x: x[1].vote,reverse=True):
                line = "\t{:.4f},{:.4f},{:.2e},{} ({:.4f},{})\n".format(peak.mass,peak.rt,peak.intensity,transformation,transmass,transformation.vote)
                f.write(line)
            f.write('\n')
            
    temp_groups = sorted(groups[filename],key = lambda x:x.M)
    outfile = outpre + '_by_M.txt'
    
    with open(outfile,'w') as f:
        for i,group in enumerate(temp_groups):
            line = "vote: {}, M: {}\n".format(group.vote,group.M)
            f.write(line)
            head_line = '\tPeak m/z,Peak rt,Peak intensity,transformation (transformed mass,vote)\n'
            f.write(head_line)
            for (peak,transformation,transmass) in sorted(group.members,key = lambda x: x[1].vote,reverse=True):
                line = "\t{:.4f},{:.4f},{:.2e},{} ({:.4f},{})\n".format(peak.mass,peak.rt,peak.intensity,transformation,transmass,transformation.vote)
                f.write(line)
            f.write('\n')

    with open(outpre + '_matched_std.txt','w') as f:
        for mol in all_hits[filename]:
            group = all_hits[filename][mol]
            line = "{} (vote={})\n".format(mol,group.vote)
            f.write(line)
            head_line = '\tPeak m/z,Peak rt,Peak intensity,transformation (transformed mass,vote)\n'
            f.write(head_line)
            for (peak,transformation,transmass) in sorted(group.members,key = lambda x: x[1].vote,reverse=True):
                line = "\t{:.4f},{:.4f},{:.2e},{} ({:.4f},{})\n".format(peak.mass,peak.rt,peak.intensity,transformation,transmass,transformation.vote)
                f.write(line)
            f.write('\n')


    include_singletons = False
    trans_counts = {}
    tot = 0
    for tr in transformations:
        trans_counts[tr] = 0
    for group in groups[filename]:
        if not include_singletons:
            if len(group.members) == 1:
                continue
        for p,t,_ in group.members:
            trans_counts[t] += 1
            tot += 1

    with open(outpre + '_tran_counts.txt','w') as f:
        for tr in sorted(transformations,key = lambda x: x.vote, reverse=True):
            line = "{},{},{:.4f}\n".format(tr,trans_counts[tr],trans_counts[tr]/(1.0*tot))
            f.write(line)
            
    # output the counts of particular adducts / fragments
    frag_counts = {}
    adduct_counts = {}
    adduct_tot = 0
    frag_tot = 0
    for tr in transformations:
        for f in tr.fragments:
            if not f in frag_counts:
                frag_counts[f] = 0
        for a in tr.adducts:
            if not a in adduct_counts:
                adduct_counts[a] = 0
                
    for group in groups[filename]:
        for p,t,_ in group.members:
            for f in t.fragments:
                frag_counts[f] += 1
                frag_tot += 1
            for a in t.adducts:
                adduct_counts[a] += 1
                adduct_tot += 1
                
    with open(outpre + '_adduct_counts.txt','w') as f:
        for a in adduct_counts:
            line = "{},{},{:.4f}\n".format(a,adduct_counts[a],adduct_counts[a]/(1.0*adduct_tot))
            f.write(line)
    
    with open(outpre + '_fragment_counts.txt','w') as f:
        for fr in frag_counts:
            line = "{},{},{:.4f}\n".format(fr,frag_counts[fr],frag_counts[fr]/(1.0*frag_tot))
            f.write(line)

            

Find the counts of particular fragments

In [56]:
frag_counts = {}
adduct_counts = {}
tot = 0
for tr in transformations:
    if len(tr.fragments)>0:
        if not tr.fragments[0] in frag_counts:
            frag_counts[tr.fragments[0]] = 0
    for a in tr.adducts:
        if not a in adduct_counts:
            adduct_counts[a] = 0

filename = files[0][1]
for group in groups[filename]:
    for p,t,_ in group.members:
        if len(t.fragments)>0:
            frag_counts[t.fragments[0]] += 1
        for a in t.adducts:
            adduct_counts[a] += 1

In [57]:
print frag_counts,adduct_counts


{'H2O': 471, 'CO2': 306, 'CO': 325} {'CH2O2': 359, '-H': 7923, 'NH3': 353, 'ACN': 295}
